<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project: "DonateFoodGoWhere" - Chatbot for donating specific food items in Singapore

https://donatefoodgowhere.streamlit.app/

## Contents:
- [Problem Statement](#Problem-Statement)
- [About Retrieval Augmented Generation (RAG) and Fine-tuning](#About-Retrieval-Augmented-Generation-(RAG)-and-Fine-tuning)
- [RAG](#RAG)
- [Fine-tuning](#Fine-tuning)
- [Evaluation](#Evaluation)

## Problem Statement

In 2022, Singapore generated 813 million kg of food waste, accounting for 11% of total waste generated. Annually, each household throws away \\$258 worth of food, equivalent of 52 plates of nasi lemak (assuming each plate cost \\$5). The amount of food waste has grown by 20% over the past 10 years, and is expected to rise further. At current rate of waste disposal, Singapore will need a new incineration plant every 7-10 years, and a new landfill every 30-35 years. 

Households contribute around half of the food waste generated. As part of Singapore’s Zero Waste Masterplan, one key component of food waste management strategies is encouraging members of public to donate excess food. Organisations have specific wish list of food items and donation requirements, and it is time-consuming for individuals to find the right organisation for the food they wish to donate.

This project aims to explore how we can help link individuals up with organisations, by developing a chatbot for individuals to enquire about donating specific food items, and find out where and how to donate, along with the relevant donation instructions. 



## About Retrieval Augmented Generation (RAG) and Fine-tuning

In this project, I used the architecture that powers ChatGPT, a generative AI tool that has revolutionized the way users get answers to their questions. To build a custom chatbot using ChatGPT's Large Language Model (LLM), two techniques, Retriever Augmented Generation (RAG) and model Fine-tuning is used. 

RAG adjusts knowledge the LLM has access to through external knowledge retrieval, while fine-tuning adjusts the behaviour of the LLM for specific domains by training it on specific dataset.

## Import libraries, API

In [1]:
from llama_index import Document, GPTVectorStoreIndex, ServiceContext
from llama_index import SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator

import os
import openai

In [2]:
os.environ["OPENAI_API_KEY"] = "sk-xxx" # replace with your API key
openai.api_key = os.environ["OPENAI_API_KEY"]

## RAG

### Load the data

[LlamaIndex's documentation](https://gpt-index.readthedocs.io/en/latest/index.html)


LlamaIndex loads data via data connectors. The easiest and most commonly used data connector is the `SimpleDirectoryReader`, which creates documents out of every file in the given directory, 'webpages' in this case. 'webpages' consists of data of various charities saved from their respective websites in PDF and html format.
<br>A `Document` is a collection of text data and metadata about that data.

In [4]:
filename_fn = lambda filename: {'file_name': filename}
reader = SimpleDirectoryReader('webpages', exclude_hidden=True, file_metadata=filename_fn)
docs = reader.load_data()
print([x.doc_id for x in docs])
print(f"Loaded {len(docs)} docs")

['783400e9-f868-4396-8136-04e0273ba75d', '466f67f2-0e62-4915-9fdd-cb3ac8ce4feb', '3dc45ba5-6383-4446-86b7-4c71ff2fd71a', '0e1064c9-1c57-4891-985f-70895abfd3bb', '5b18db31-9887-46c8-9930-a8052f679881', '3b8f3aa4-579a-456e-ae2f-3f2b6cd1394f', '8d3a28b0-a692-47b3-b1d3-b11b58d6d5be', '54a2533d-25be-4504-a9b6-0e4548c7d92b', '6cfdc969-b40c-446c-bd5e-419f1cfb68d7', '04ad98c5-2a79-491f-a496-71d1fd7624a0', '0cba5ae9-654a-408c-8c9f-eb8fdbbe4927', 'a4c9e7bf-586a-49b1-b637-3c85aa667dd0', '55e10171-5cf0-4757-b980-9173aac1603e', '736fa0fb-fbb6-44c3-86cb-0bbf9dc25daf', '4fc1d031-64f7-451e-92b7-b7c9748b38fa', '436e66c2-17ca-4f7a-b40d-45ae065469d0', 'd0d7589f-e02b-4d70-a0c6-974d746a0ebc', '7e9cb513-3bf3-4a62-86ed-77f5c319a803', 'b13336e5-29d2-49f9-acd4-e1b67ab7f25f', '7ea6ff6d-6cf7-4818-92b1-a8d961f88806', '273ad003-0702-4e84-a1df-a34aa39e6757', 'd4a611e1-038f-4175-98bd-6c14a22e63f1', '52db3589-80e6-4227-a74e-548a7db0b65b', 'ba01ddc3-89d3-4d2c-9af7-a6f81718e95f', 'ee9edb13-8af2-4602-8c01-d20cd07310e1',

### Indexing and Storage

With all the data loaded, a list of 70 Document objects is created. Proceed to build an Index over these objects, which makes it ready for querying by an LLM. There are 4 types of indexing: Summary index, VectorStore Index, Tree Index and Keyword Table Index. 

In this project, `VectorStoreIndex` is used as it is by far the most frequent type of indexing. It takes the Documents and splits them up into Nodes, then creates `vector embeddings` of the text of every node. `Vector embedding` aka embedding is a numerical representation of the semantics, or meaning of the text. Two pieces of text with similar meanings will have mathematically similar embeddings, even if the actual text is quite different. This mathematical relationship enables semantic search, where a user provides query terms and LlamaIndex can locate text that is related to the meaning of the query terms rather than simple keyword matching. This is a big part of how Retrieval-Augmented Generation works.

Definition of classes:
- `ServiceContext` is a bundle of configuration data which can be passed to other stages of the pipeline.

In [6]:
# Instantiate the LLM (gpt-3.5-turbo) from OpenAI and pass it to ServiceContext().
# The GPTVectorStoreIndex will use gpt-3.5-turbo as embedding model to index the documents
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0)) # degree of randomness from 0 to 1.  
index = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context)

After indexing, the ouput is stored in disk using the built-in .persist() method to avoid the time and cost of having to re-index it.

In [7]:
index.storage_context.persist(persist_dir="data/index.vecstore")

## Fine-tuning

Currently, there are three key integrations with [LlamaIndex for fine-tuning](https://gpt-index.readthedocs.io/en/latest/optimizing/fine-tuning/fine-tuning.html).

In this project, since GPT-3.5-Turbo is used, I will try to distill a better model (e.g. GPT-4) into the simpler/cheaper model (e.g. GPT-3.5), i.e. finetuning GPT-3.5-turbo to ouput GPT-4 responses. 

The key steps are:
1) Split the documents into train/evaluation set
1) Generate a question/answer dataset over the train set
    - use GPT-3.5-Turbo to generate questions from the external data, and GPT-4 query engine to generate answers.
    - `OpenAIFineTuningHandler` callback automatically logs questions/answers to a dataset.
2) Launch a finetuning job with `OpenAIFinetuneEngine`, and get back a finetuned model
3) Evaluate the performance of the finetuned model and compare with the base model


### Generate Train Dataset

This dataset is for finetuning the base model.

In [8]:
# Shuffle the documents
import random

random.seed(42)
random.shuffle(docs)

gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)

In [9]:
# To avoid RuntimeError: asyncio.run() cannot be called from a running event loop
# The below code is to unblock: nest the event loops

import nest_asyncio
nest_asyncio.apply()

In [10]:
question_gen_query = (
    "You are an individual food donor looking to donate specific food items. \
    Your task is to setup a quiz or examination. \
    Using the provided context from documents on different food support organisations, \
    formulate a single question that captures an important fact from the context. \
    Restrict the question to the context information provided."
)

# find out more about question generation from 
# https://gpt-index.readthedocs.io/en/latest/examples/evaluation/QuestionGeneration.html

dataset_generator = DatasetGenerator.from_documents(
    docs[:40],
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

In [11]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


In [12]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

![train_questions.png](train_questions.png)

### Generate Evaluation Dataset

This dataset is for subsequent evaluation step to measure the performance of the models.
<br> Questions are generated from a different set of documents.

In [13]:
dataset_generator = DatasetGenerator.from_documents(
    docs[
        40:
    ],  # since we generated question for the first 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

In [14]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  31  questions


In [15]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

![Screenshot of eval generated](eval_questions.png)

### GPT-4 to Generate Training Data

In [32]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [33]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [34]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [35]:
for question in questions:
    response = query_engine.query(question)

### Create `OpenAIFinetuneEngine`

`OpenAIFinetuneEngine` is a finetune engine that will take care of launching a finetuning job, and returning an LLM model that can be directly plugged in to the rest of LlamaIndex workflows.

In [36]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 43 examples to finetuning_events.jsonl


In [37]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events.jsonl"
)

In [38]:
finetune_engine.finetune()

Num examples: 43
First example:
{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}
{'role': 'user', 'content': "Context information is below.\n---------------------\npage_label: 8\nfile_name: webpages/foodfromtheheart.pdf\n\n30/10/2023, 14:19 In-Kind Food Donation in Singapore | Food from the Heart\nhttps://www.foodfromtheheart.sg/in-kind-donations 4/5Evaporated/condensed milk\nCooking oil, 500ml or 1L\nBread spread\n*Priority items\n\xa0\nCommunity Shops Wish List ( )\nCoﬀee in Sachets (All ﬂavours)\nTea in Sachets (All ﬂavours)\nMilo 3-in-1\nAssorted canned pork products\nAssorted Biscuits\nCanned Fruits\nCanned fried dace/ Sardines\nRice 2.5kg - 5kg\

![notification of successful finetuned job](finetune_job.png)

In [42]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-TELtB14BEZObAnOi4GnuilS9 at 0x7ff3e8fc0bd0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-TELtB14BEZObAnOi4GnuilS9",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698723349,
  "finished_at": 1698724106,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::8Fa1rB0k",
  "organization_id": "org-uKgXyGnBOIt0jbTHoR4VVtz2",
  "result_files": [
    "file-KYK95HQQBYuOJhMXJ1gOIe2r"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-MrYX9OR6MpZcHNDODBOOGXnd",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 126126,
  "error": null
}

In [43]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0)

## Evaluation

To measure the performance of the pipeline, whether it is able to generate relevant and accurate responses given the external data source and a set of queries, we use 2 evaluation metrics from [`ragas` evaluation library](https://github.com/explodinggradients/ragas/tree/main/docs/concepts/metrics). Ragas uses LLMs under the hood to compute the evaluations.

The performance of the base model, gpt-3.5-turbo, will be compared with the fine-tuned model.

Computation of evaluation metrics require 3 components: 
1) `Question`: A list of questions that could be asked about my external data/documents, generated using .generate_questions_from_nodes in above fine-tuning step<br>
2) `Context`: Retrieved contexts corresponding to each question. The context represents (chunks of) documents that are relevant to the question, i.e. the source from where the answer will be generated.<br>
3) `Answer`: Answer generated corresponding to each question from baseline and fine-tuned model.

The two metrics are as follow:

- `answer_relevancy` - Measures how relevant the generated answer is to the question, where an answer is considered relevant when it <u>directly</u> and <u>appropriately</u> addresses the orginal question, i.e. answers that are complete and do not include unnecessary or duplicated information. The metric does not consider factuality. It is computed using `question` and `answer`, with score ranging between 0 and 1, the higher the score, the better the performance in terms of providing relevant answers. To calculate this score, the LLM is prompted to generate an appropriate question for the generated answer multiple times, and the mean cosine similarity between these generated questions and the original question is measured. The underlying idea is that if the generated answer accurately addresses the initial question, the LLM should be able to generate questions from the answer that align with the original question, i.e. high mean cosine similarity, translating to high score.


- `faithfulness` - Measures how factually accurate is the generated answer, i.e. if the response was hallucinated, or based on factuality (from the context). It is computed from `answer` and `context`, with score ranging between 0 and 1, the higher the score, the better the performance in terms of providing contextually accurate information. To calculate this score, the LLM identifies statements within the generated answer and verifies if each statement is supported by the retrieved context. The process then counts the number of statements within the generated answer that can be logically inferred from the context, and dvide by the total number of statements in the answer. 

Additional note: Cosine similarity is a metric used to measure how similar two items are. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. The output value ranges from 0–1 where 0 means no similarity, whereas 1 means that both the items are 100% similar.
<br>Hallucinations refer to instances where the language model produces information or claims that are not accurate or supported by the input context.

Resources:
<br>https://cobusgreyling.medium.com/rag-evaluation-9813a931b3d4
<br>https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/
<br>https://medium.aiplanet.com/evaluate-rag-pipeline-using-ragas-fbdd8dd466c1

### Evaluation of base model: GPT-3.5-Turbo

In [5]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [6]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0), context_window=2048
)

index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

# as_query_engine builds a default retriever and query engine on top of the index
# We configure the retriever to return the top 2 most similar documents, which is also the default setting
query_engine = index.as_query_engine(similarity_top_k=2)

In [7]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [8]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [faithfulness, answer_relevancy])
print(result)

evaluating with [faithfulness]


100%|████████████████████████████████████████████| 3/3 [20:48<00:00, 416.04s/it]


evaluating with [answer_relevancy]


 33%|███████████████                              | 1/3 [00:56<01:52, 56.25s/it]Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
100%|████████████████████████████████████████████| 3/3 [21:58<00:00, 439.65s/it]


{'ragas_score': 0.9167, 'faithfulness': 0.8692, 'answer_relevancy': 0.9697}


In [9]:
df_gpt_35 = result.to_pandas()

# Export cleaned dataframe as .csv
df_gpt_35.to_csv("eval/df_gpt_35.csv",index=False)

### Evaluation of fine-tuned model

Run the fine-tuned model on the evaluation dataset again to measure any performance increase.

In [10]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [14]:
# pass in ft_llm directly into ServiceContext
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

index = VectorStoreIndex.from_documents(docs, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [15]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [16]:
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [faithfulness, answer_relevancy])
print(result)

evaluating with [faithfulness]


100%|████████████████████████████████████████████| 3/3 [25:10<00:00, 503.58s/it]


evaluating with [answer_relevancy]


100%|████████████████████████████████████████████| 3/3 [11:56<00:00, 238.94s/it]


{'ragas_score': 0.9255, 'faithfulness': 0.8779, 'answer_relevancy': 0.9784}


In [17]:
df_ft_llm = result.to_pandas()

# Export cleaned dataframe as .csv
df_ft_llm.to_csv("eval/df_ft_llm.csv",index=False)

| Pipeline                    | Answer Relevancy | Faithfulness |
|-----------------------------|------------------|--------------|
| RAG+GPT-3.5-Turbo           | 0.9697           | 0.8692       |
| RAG+Finetuned GPT-3.5-Turbo | 0.9784           | 0.8779       |


Both models are high in performance for both metrics, with the fine-tuned model having an improvement of 1%. 
Instead of fine-tuning, we could get comparable performance using the base model, which involves far less computing resource. 
And on top of that, as food items for organisation’s wishlist change over time, RAG alone can easily and quickly adapt to the new data. As such, RAG+GPT-3.5-Turbo is chosen for deployment. 

## Exploring Differences

Let's quickly compare the differences in responses, to demonstrate that fine tuning did indeed change something.

In [18]:
index = VectorStoreIndex.from_documents(docs)

In [19]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [28]:
print(questions[12])

What is the contact information for donating dry rations to YWCA Singapore?


### Original

In [29]:
from llama_index.response.notebook_utils import display_response

query_engine = index.as_query_engine(service_context=gpt_context)

response = query_engine.query(questions[12])

print(response)

To donate dry rations to YWCA Singapore, you can contact Irene at 6223 1227 or email csp@ywca.org.sg. They will provide you with further information on how to fulfill the wishlist items and make your generous contributions.


### Fine-tuned

In [30]:
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

In [32]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

print(response)

The contact person for donating dry rations to YWCA Singapore is Irene, and she can be reached at 6223 1227 or via email at csp@ywca.org.sg.


The original base model generated additional sentence "They will provide you with further information on how to fulfill the wishlist items and make your generous contributions." in the answer. This is considered unnecessary information as it is not directly related to the query. The generated answer by the fine-tuned model is concise and sufficiently informative.

[Click for app code in streamlit](../streamlit/foodbot.py)